---
title: "CSX46 Class Notebook 5 - Components and Eulerian Paths"
author: "Trent VanHawkins"
date: today
date: today
format: 
    pdf:
        geometry: "margin=0.75in"
        mathspec: true
        code-overflow: wrap
        cap-location: bottom
        tbl-cap-location: bottom
        header-includes: 
        - \usepackage{fancyhdr, amsthm, amssymb,amsfonts,amsthm, amsmath, bbm}
        - \usepackage{float, tabularx}
        - \floatplacement{table}{H}
        - \pagestyle{fancy}
        - \fancyhead[R]{Notebook 5}
        - \fancyhead[L]{Trent VanHawkins}
        - \fancyfoot[C]{\thepage} # Center page number at bottom of each page
page-layout: full
execute: 
  eval: true
---

In this notebook we are going to find the number of proteins that are in the giant component of the (undirected) human protein-protein interaction network (Pathway Commons 2, release 9), using igraph.

We are going to need igraph, pandas, numpy, collections, pprint, and operator; see earlier notebooks for how to install them


In [1]:
import igraph
import pandas as pd
import numpy as np
import collections
import pprint
import operator
from typing import List, Set

We will need to download the Pathway Commons interactions file in Simple Interaction Format (SIF) and uncompress it (see notebooks 1-4)

In [2]:
# 2. Download the file into that specific relative path
!curl https://csx46.s3-us-west-2.amazonaws.com/PathwayCommons9.All.hgnc.sif.gz \
    --output ../DataRaw/PathwayCommons9.All.hgnc.sif.gz

# 3. Unzip it in place
!gunzip -f ../DataRaw/PathwayCommons9.All.hgnc.sif.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5930k  100 5930k    0     0  5743k      0  0:00:01  0:00:01 --:--:-- 5746k


Step 1:  load the SIF file into a data frame `sif_data`, using the `pandas.read_csv` function, and name the columns `species1`, `interaction_type`, and `species2`.

In [3]:
# Use the relative path to go up one level and into DataRaw
file_path = "../DataRaw/PathwayCommons9.All.hgnc.sif"

sif_data = pd.read_csv(file_path, 
                       sep="\t", 
                       names=["species1", "interaction_type", "species2"])

# Display the first few rows to confirm it loaded correctly
sif_data.head()

,species1,interaction_type,species2
0,A1BG,controls-expression-of,A2M
1,A1BG,interacts-with,ABCC6
2,A1BG,interacts-with,ACE2
3,A1BG,interacts-with,ADAM10
4,A1BG,interacts-with,ADAM17


Step 2:  restrict the interactions to protein-protein undirected ("in-complex-with", "interacts-with"), by using the `isin` function and then using `[` to index rows into the data frame.  Call the returned data frame `interac_ppi`.

In [4]:
interaction_types_ppi = set(["interacts-with",
                             "in-complex-with"])
interac_ppi = sif_data[sif_data.interaction_type.isin(interaction_types_ppi)].copy()

Step 3: restrict the data frame to only the unique interaction pairs of proteins (ignoring the interaction type), and call that data frame `interac_ppi_unique`. Make an igraph `Graph` object from `interac_ppi_unique` using `Graph.TupleList`, `values`, and `tolist`. Call `summary` on the `Graph` object. Refer to the notebooks for the in-class exercises in Class session 3.

In [5]:
boolean_vec = interac_ppi['species1'] > interac_ppi['species2']
interac_ppi.loc[boolean_vec, ['species1', 'species2']] = interac_ppi.loc[boolean_vec, ['species2', 'species1']].values

interac_ppi_unique = interac_ppi[["species1", "species2"]].drop_duplicates()


ppi_igraph = igraph.Graph.TupleList(interac_ppi_unique.values.tolist(), directed=False)
igraph.summary(ppi_igraph)

IGRAPH UN-- 17531 475553 -- 
+ attr: name (v)


Step 4: Map the components of the network using the `igraph.Graph.connected_components` method. That method returns a `igraph.clustering.VertexClustering` object.  Call the `sizes` method on that `VertexClustering` object, to get a list of sizes of the components.  Print the component sizes list.  What is size (cardinality) of the giant component, i.e., the largest component?

In [6]:
# call the `connected_components` method on the `ppi_igraph` object, and assign the
# resulting `VertexClustering` object to have object name `ppi_components`
ppi_components = ppi_igraph.connected_components()

# call the `sizes` method on the `ppi_components` object, and assign the
# resulting list object to have the name `ppi_component_sizes`.
ppi_component_sizes = ppi_components.sizes()
print("component sizes, in order of component ID: ")
print( ppi_component_sizes )

# find its maximum value using the `max` built-in function
print(f"maximum component size: {max(ppi_component_sizes)}")


component sizes, in order of component ID: 
[17524, 2, 2, 3]
maximum component size: 17524


In the `VertexClustering` object returned by the call to `clusters`, each vertex has a component ID which is an integer. How do we get the component ID numbers of the vertices?  We get them from the `membership` variable in the `VertexClustering` object, as a python `list`. Let's inspect the component IDs of the first six vertices:

In [7]:
component_ids = ppi_components.membership

print(component_ids[0:6])

[0, 0, 0, 0, 0, 0]


Now, let's manually compute the sizes (cardinalities) of the components and compare them to the sizes we get from the `VertexClustering.sizes` method. They should be the same.

In [19]:

import enum
from termios import CSIZE


ctr = collections.Counter(component_ids)
print("from our Counter based counting: ")
print(ctr)
print("from calling VertexClustering.sizes: ")
print({cid: csize for cid, csize in enumerate(ppi_component_sizes)})


from our Counter based counting: 
Counter({0: 17524, 3: 3, 1: 2, 2: 2})
from calling VertexClustering.sizes: 
{0: 17524, 1: 2, 2: 2, 3: 3}


What if we wanted to know the IDs of the components, in decreasing order of component size?  (i.e., giant component ID first, then decreasing). So, let's print the indexes in `ppi_components` in reverse sorted order. Can you do it with `numpy.argsort`?  

In [ ]:
list(np.argsort(-np.array(ppi_component_sizes)))

[np.int64(0), np.int64(3), np.int64(1), np.int64(2)]


Can you do the same thing using `operator.itemgetter(1)` and `sorted` (with `reverse=True`)?

In [41]:
[id for id, _ in sorted(ctr.items(), reverse=True, key=operator.itemgetter(1))]

[0, 3, 1, 2]

Can you get the names of the proteins (labels of vertices) that are in the component with three proteins?
You may want to do this in two steps. First, get the indexes (`three_component_inds`) of the vertices for which the component ID is == 3. You may want to call `enumerate` with `component_ids` as the agument, and use list comprehension. Then, call the `vs` method on `ppi_igraph` and pass the `three_component_inds` list as the argument. In the resulting object, obtain the `name` attributes by indexing, like `["name"]`.

In [53]:
three_component_inds = [idx for idx, val in enumerate(component_ids) if val ==3]
ppi_igraph.vs(three_component_inds)["name"]

['TAS1R1', 'TAS1R3', 'TAS1R2']

Can you write a function `is_eulerian_path` that checks a vertex sequence to see if it is a Eulerian path of a given simple undirected graph (represented in adjacency set format)?

In [55]:
# define a helper function to make an "edge key"
# (as a string) from the min(n,m) and max(n,m), separated by a hyphen
from typing import Counter


def make_edge_key(n: int, m: int) -> str:
  return  str(min(n,m)) + '-' + str(max(n,m))

def is_eulerian_path(graph: List[Set], path: List[int]) -> bool:
    # create a dictionary `edge_counts` to hold the count of each edge "key"
    edge_counts = dict()

    # iterate over enumerate(graph), assigning n to index and neighbor_set to the value
    for n, neighbor_set in enumerate(graph):
        # if n is in `neighbor_set`, this is not a simple graph; raise ValueError
        if n in neighbor_set:
            raise ValueError("This graph contains a loop and is not simple")

        # iterate over neighbors m of n:
        for m in neighbor_set:
            # make an "edge key" (as a string) using the `make_edge_key` function
            edge_key = make_edge_key(n, m)
            # for that edge key, set the value in `edge_counts` to 0 (zero)
            edge_counts[edge_key] = 0
    # iterate over all indexes of entries in the path, except the last:
    for i in range(0, len(path) - 1):
        # get the starting vertex for this edge
        n = path[i]
        # get the ending vertex for this edge
        m = path[i + 1]
        # construct the edge key, as above
        edge_key = make_edge_key(n, m)
        # increment the counter for this edge key
        edge_counts[edge_key] += 1
    # every value in the counter dictionary should be exactly 1
    return set(list(edge_counts.values())) == {1}

print(is_eulerian_path([{1,2},{0},{0}], [2, 0]))
print(is_eulerian_path([{1,2},{0},{0}], [2, 0, 1]))


False
True


Code-spellunking question:  go to the GitHub repo for igraph (https://github.com/igraph), and find the code module `components.c`.  For the weakly connected components, is it doing a breadth-first search (BFS) or a depth-first search (DFS)?

*On the github I found a fuction called `igraph_i_connected_components_weak`. I do not know how to read programs in C, but it appears to be doing a breadth-first search (BFS) since it is usina a queu as opposed to a stack to search through the graph.*